### Description:

AES-128 algorithm with a hardcoded key is available as service.

To perform encryption, the binary requires 16-bytes plaintext (in hex):

```
    ../wbcenc --ptext="010203040506070809000a0b0c0d0e1f"
```    
The result is printed to console:
```
    AA86E77B831376EB5177C26DDC0C1632 074EBE285954C25C0C3E71C7EFE30AB
```    
where:
```
    AA86E77B831376EB5177C26DDC0C1632 is a ciphertext
    
    074EBE285954C25C0C3E71C7EFE30AB is a trace (information left by a programmer)
```
### Tips:

A 'trace' is a side-channel information left by a programmer to debug the implementation.

In this example a trace is a 10th round input state of the AES-128 algorithm, i.e., a State before the Sbox operation in the last round.

### Task:

Your task is to find the Master key (round key 0) embedded into the binary. 

The master key is in the form of HEIG{XXXXXXXXXX}, where X is an ASCII printable symbol.

### Leakage illustration

<img src="support/Slide.png">

In [ ]:
import numpy as np
import binascii
import string
import re
import socket

HOST = 'iict-mv330-sfa'
PORT = 4000

import sca_training
#----------------------------------------------------------------------------
# This function calls tested binary ../wbcenc either with a user-defined 
# plaintext (if plaintext satisfies all the requirements) or with a 
# predefined plaintext (the same as in the header above)
#
# INPUTS:
#     plaintext - a string of 32 symbols representing 16 hex bytes of ciphertext
#     verbose   - a flag to print values in the function call or not
# OUTPUTS:
#     output  - a raw binary output
#     ctext   - resulted ciphertext converted to numpy array of uint8
#     trace   - a trace associated with encryption process
#----------------------------------------------------------------------------
def binary_aes128_encrypt(plaintext, verbose=False):
    import subprocess
    pattern = '[0-9A-F]{32}'
    
    #Check the correctness of the plaintext
    if all(c in string.hexdigits for c in plaintext) and len(plaintext) == 32:
        
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.connect((HOST, PORT))
            s.recv(1024)
            s.sendall(plaintext.encode())
            s.sendall(b'\n')

            output = s.recv(1024)
    #TODO: parse incoming data
        
    #Get the ciphertext (first 32 symbols) and the trace (last 32 symbols)
    result = re.findall(pattern, str(output))
    #print(result[0], result[1])
    
    #Transform the result into numpy array
    ctext = np.frombuffer(binascii.unhexlify(result[0]), dtype=np.uint8)
    trace = np.frombuffer(binascii.unhexlify(result[1]), dtype=np.uint8)

    if (verbose):
        print('Binary output:', output.strip())
        print('Ciphertext as numpy array:', ctext)
        print('Trace as numpy array:', trace)
    
    return output, ctext, trace


#----------------------------------------------------------------------------
# This function calls supportive library to compute a master key from
# the last round key.
#
# INPUTS:
#     last_round_key - a numpy array of 16 elements representing 16 hex bytes of
#                      the last round key, i.e., Round Key 10
# OUTPUTS:
#     key_schedule  - 11 round keys of 16 bytes each (key_schedule[0,0,:] is the
#                     master key
#----------------------------------------------------------------------------
# key_schedule = sca_training.inverse_key_expansion(last_round_key)

In [ ]:
output, ctext, trace = binary_aes128_encrypt("010203040506070809000a0b0c0d0e1f", verbose=True)

In [34]:
import numpy as np
import sca_training

cipher = np.array(sca_training.hex_int("AA86E77B831376EB5177C26DDC0C1632"), dtype=np.uint8)
tenth = np.array(sca_training.hex_int("074EBE285954C25C0C3E71C7EFE30AB6"), dtype=np.uint8)

tenth = sca_training.sbox(tenth)
tenth = sca_training.shift_rows(tenth)

mk = np.array([np.bitwise_xor(c, t) for (c, t) in zip(tenth, cipher)])
mk = sca_training.inverse_key_expansion(mk)
print(mk[0][0])
''.join(chr(c) for c in mk[0][0])

[ 72  69  73  71 123  83 105 109 112 108 101  32  65  69  83 125]


'HEIG{Simple AES}'

### ATTACK CODE
Here you need to implement your attack which gives you the last round key.

Once the last round key is found - you need to compute the master key: this can be done with the function get_master_key()

### Programming tips
#### Numpy code
* np.arange(256).astype(np.uint8)
* np.bitwise_xor(m1, m2)
* np.where()

#### Code prepared for training
* sca_training.invSbox[sbox_out]
* sca_training.shift_rows(trace)

#### Various cod
* ''.join('{:02x}'.format(c) for c in int_array)